<a href="https://colab.research.google.com/github/UrviVassisht15/sentiment-analysis-with-transformers/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing required packages


In [1]:
!pip install torch
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00


## Importing necessary libraries to setup machine learning enviornment

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import pipeline
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import pipeline
from datasets import load_dataset
from sklearn.model_selection import train_test_split

## Loading and Splitting Dataset

Here we load a IMDb sentiment data, divides it into training/testing texts and labels via `train_test_split`, allocating 20% for testing.

In [3]:
dataset = load_dataset("imdb")
train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset["train"]["text"], dataset["train"]["label"], test_size=0.2, random_state=42
)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

##Tokenization and Dataset Creation

This section uses a BERT tokenizer for `train_texts` and `test_texts`, creating a `SentimentDataset` class to encapsulate tokenized data and labels for sentiment analysis.

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

##DataLoader Initialization

The commands create `train_loader` and `test_loader` to organize data into batches for training and testing. `train_loader` shuffles data for training, `test_loader` maintains order for accurate testing, simplifying model learning and accuracy assessment.

In [5]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

##BERT Model Loading and Optimization Setup

Here we set up a BERT model for sequence classification, pre-trained on "bert-base-uncased," configured to classify into two labels. It uses AdamW optimizer with a learning rate of 5e-5 for training.

In [6]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


##Model Training

The code prepares the device for computation, executes a training loop, and iterates through batches. It optimizes the model by calculating loss, backpropagating, and updating parameters to improve sentiment label predictions.

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

##Model Evaluation on Test Data:

In the following section we evaluate the model, computes predictions on the test set, tracks correct predictions and total samples, then calculates and prints the test set accuracy.

In [8]:
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        total_correct += torch.sum(predictions == labels)
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f"Accuracy on test set: {accuracy.item() * 100:.2f}%")

Accuracy on test set: 85.76%


##Sentiment Analysis with Pipeline:

this section uses pipeline to analyze textual inputs, providing assessments on the positivity or negativity of the content along with confidence levels.

In [9]:
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
result = sentiment_pipeline("I love using transformers for sentiment analysis!")
print(result)

[{'label': 'LABEL_0', 'score': 0.7732833027839661}]
